# Building a Database for Crime Reports


## Creating the Database and the Schema


In [1]:
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True
cur = conn.cursor()
# create the crime_db database
cur.execute("CREATE DATABASE crime_db;")
conn.close()

DuplicateDatabase: database "crime_db" already exists


In [2]:
# now the crime_db database exists to we can connect to it
conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cur = conn.cursor()
# create he crimes schema
cur.execute("CREATE SCHEMA crimes;")

DuplicateSchema: schema "crimes" already exists


## Obtaining the Column Names and Sample
 

In [3]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

## Creating a Function for Analyzing Column Values


In [4]:
def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Analyzing the Maximum Length of the Description Column


In [5]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [6]:
descriptions = get_col_set("boston.csv", 2) # description is at index number 2
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


## Creating the Table


In [7]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


We will use the same names for the column headers.

The number of different values of each column was the following:

```
incident_number 298329
offense_code       219
description        239
date	          1177
day_of_the_week      7
lat              18177
long	         18177
```

From the result of printing `first_row`, we see which kind of data we have:

```
integer numbers
integer numbers
string
date
string
decimal number
decimal number
```

Only column `day_of_the_week` has a small range of values, so we will only create an enumerated datatype for this column. Column `offense_code` is also a good candidate since there is probably a limited set of possible offense codes.

We saw that the `offense_code` column has size at most 59. To be safe, we will limit the size of the description to 100 and use the `VARCHAR(100)` datatype.

The `lat` and `long` columns need to hold quite a lot of precision, so we will use the `decimal` type.

In [8]:
# Create the enumerated datatype for representing the weekday.
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# Create the table.
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

DuplicateObject: type "weekday" already exists


## Load the Data into the Table


In [9]:
# Load the data from boston.csv into the table boston_crimes that is in the crimes schema.
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")
# Print the number of rows to ensure that they were loaded.
print(len(cur.fetchall()))

UniqueViolation: duplicate key value violates unique constraint "boston_crimes_pkey"
DETAIL:  Key (incident_number)=(1) already exists.
CONTEXT:  COPY boston_crimes, line 2


## Revoke Public Privileges


In [10]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating the Read Only Group


In [11]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

DuplicateObject: role "readonly" already exists


## Creating the Read Write Group


In [12]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

DuplicateObject: role "readwrite" already exists


## Creating One User for Each Group



In [13]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

DuplicateObject: role "data_analyst" already exists
